In [1]:
#mil python
# install dependencies 
# clean up the fucking code

# Initialization
import numpy as np
import matplotlib.pyplot as pl
from scipy import interpolate
from scipy import stats
import healpy as hp  # NOTE: you can neglect warning about version mismatch in CFITSIO if necessary 

N_side = 2048      # Both maps will have this N_side. Healpix map has 12 * N_side**2 pixels

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy import units as u

import pandas as pd
%matplotlib inline

In [ ]:
# maps maps maps maps

# Planck 2018 CIB map: http://pla.esac.esa.int/pla/aio/product-action?MAP.MAP_ID=COM_CompMap_CIB-GNILC-F545_2048_R2.00.fits
CIB_map = hp.read_map('COM_CompMap_CIB-GNILC-F545_2048_R2.00.fits')        # read CIB map
CIB_mask = hp.read_map('/Users/sophiarisin/Dropbox/Mac (2)/Downloads/cib_mask.fits')                                    # read CIB mask

CIB_masked = hp.ma(CIB_map)
CIB_masked.mask = np.logical_not(CIB_mask)
hp.mollview(CIB_masked)

In [ ]:
def read_nobs_pyfits(filename):
    with fits.open(filename, memmap=True) as hdul:
        data = (hdul[1].data)
        return np.shape(data)[0], hdul[1].columns.names

def read_test_pyfits(filename, colname):
    with fits.open(filename, memmap=True) as hdul:
        data = (hdul[1].data[colname])
        return data.copy()

def get_BOSS_data(gal):
    nObs, cols = read_nobs_pyfits(gal)
    colnames = [x for x in cols if x in ['ID', 'RA', 'DEC', 'Z', 'NZ', 'BOSS_SPECOBJ_ID',
                                         'BOSS_TARGET1', 'BOSS_TARGET2', 'EBOSS_TARGET0', 'ZOFFSET', 'TARGETOBJID',
                                         'OBJID', 'PLUG_RA', 'PLUG_DEC', 'Z']]
    ncols = len(colnames)
    myGalaxy = pd.DataFrame(data=np.zeros([nObs, ncols]), columns=colnames)
    for rowname in myGalaxy.columns:
        myGalaxy[rowname] = read_test_pyfits(gal, rowname).byteswap().newbyteorder()
    print(myGalaxy.columns)
    myGalaxy = myGalaxy.sort_values(by=['Z'])
    return myGalaxy



boss_catalog_path_s = 'galaxy_DR12v5_CMASS_South.fits'
boss_catalog_path_n = 'galaxy_DR12v5_CMASS_North.fits'
# Load the relevant entries of the catalog (we will be mostly concerned with RA, DEC, and redshift)
boss_catalog_s = get_BOSS_data(boss_catalog_path_s)
boss_catalog_n = get_BOSS_data(boss_catalog_path_n)

# Load the relevant entries of the catalog (we will be mostly concerned with RA, DEC, and redshift)
datas = get_BOSS_data(boss_catalog_path_s)
datan = get_BOSS_data(boss_catalog_path_n)
datas = pd.DataFrame(datas)
datasmodd = pd.DataFrame(datas[(datas.Z > .4) * (datas.Z < .5)])
datanmodd = pd.DataFrame(datan[(datan.Z > .4) * (datan.Z < .5)])

modtot = pd.concat([datanmodd,datasmodd])
modtot